In [1]:
import json
import zipfile

import urllib3
import pandas as pd

import crawler
import helpers
from helpers import DATA_PATH

urllib3.disable_warnings()

In [2]:
with open(DATA_PATH / "processos_pautados.json", "r") as f:
    pautas = json.load(f)

In [3]:
datas_ses_presencial = []
ses_virtual = []

for pautas_mes in pautas:
    mes = pautas_mes["mes"]
    ano = pautas_mes["ano"]
    datas_ses_presencial.extend(
        [f"{str(d).zfill(2)}/{mes}/{ano}" for d in pautas_mes["diasPresenciais"]]
    )

    ses_virtual.extend(pautas_mes["julgamentosVirtuais"])

# Carregando sessões já coletadas

Essa parte assume que não há alterações nas pastas já coletadas.

In [4]:
col_pres = helpers.load_json_from_zip("pautas_presenciais")
col_virtuais = helpers.load_json_from_zip("pautas_virtuais")

Sessões presenciais são identificadas pela data. Assim, podemos remover todas as já coletadas.

In [5]:
pres_to_collect = set(datas_ses_presencial) - set([s.get("data") for s in col_pres])

Sessões virtuais são identificadas pelo conjunto da data de início e data de fim.

In [6]:
virt_to_collect = set(
    [(s.get("dataInicial", ""), s.get("dataFinal", "")) for s in ses_virtual]
) - set([(s.get("inicio", ""), s.get("fim", "")) for s in col_virtuais])

# Coletando sessões virtuais

In [7]:
pautas_virtuais = col_virtuais

In [8]:
for pauta in virt_to_collect:
    pauta = {"inicio": pauta[0], "fim": pauta[1]}
    pautas_virtuais.append(crawler.get_pauta_virtual(pauta))

In [9]:
helpers.save_json_to_zip(pautas_virtuais, "pautas_virtuais")

# coletando sessões presenciais

In [10]:
pautas_pres = col_pres

for data in pres_to_collect:
    pauta = crawler.get_pauta_presencial(data)
    pautas_pres.append(pauta)



In [11]:
helpers.save_json_to_zip(pautas_pres, "pautas_presenciais")